# DataShed Technical Test

## Requirements

The application should:

Read data from the source file
Output:

1) the number of records in the source data set

2) the number of unique records* in the source data set

3) the number of different people** in the source data set

4) Write the data that is considered duplicated/related to a CSV file called relateddata.csv

5) Consider data that is similar and classify it as a duplicated/related. For example "Wilyam Premadasta" and "William Premadasa" should be considered the same person.

*exact duplicate records should be eliminated from this measure.

** the number of records following your de-duplication processing.

## Solution - Fergus McLellan
My solution is based on Python 3, relying heavily on Pandas to import and clean the data. The "fuzzywuzzy" module is used to assist with the identification of records which are related with similar given names and surnames. The requirements give no indication as to how much weight should be given to the date of birth, and there are a number of records which have the same names, but a very different date of birth. For the purposes of this solution, it is assumed that the date of birth is correct, and we are identifying records which have the same date of birth and similar names. There is also no direction given as to whether or not any errors in the sex provided should be captured, as there are a large number of "v" entries.

Due to the use of Pandas, a Jupyter Notebook of the prototype solution is included here for convenience. However, the full solution can be found in the .py file on Github.

## Output files
<ul>
<li>summary.txt - contains the summary information and figures for the number of records and duplicates outlined in the requirements</li>
<li>exact_duplicates.csv - contains the records which have an exact duplicate</li>
<li>related.csv - contains the data that is considered related, with the related pair records listed side by side</li>
<li>cleaned.csv - contains the final data which has had duplicate and the 2nd occurence of a related record removed</li>
<li>invalid_data.csv - contains the records which did not contain data in the expected format. Some entries were identified with an inconsistent date format.</li>
</ul>


In [1]:
# Import required modules
import pandas as pd
import pandas_schema
from pandas_schema.validation import CustomElementValidation
import numpy as np
import unicodedata
import re
import os
import datetime
from fuzzywuzzy import fuzz

In [2]:
# Increase the fuzz_score to capture fewer similarities, and return more strict comparisons
# Decrease the fuzz_score to capture more vague similarities
fuzz_score = 50

# Set input file to be analysed here
input_file = 'DataShed_Technical_Test.csv'

In [3]:
# Open file and check for errors
if not os.path.exists(input_file):
    print("Unable to find filename provided")

try:
    df_people_records = pd.read_csv(input_file)

except pd.errors.EmptyDataError:
    print("Note: the given .csv was empty.")
except pd.errors.ParserError:
    print("Error parsing .csv file.")


## Requirement 1) the number of records in the source data set

In [4]:
# Capture the number of entries in the dataframe, and assert that there are more than 5 rows
no_of_records_in_source = df_people_records.shape[0]
print("Total number of records in source: " + str(no_of_records_in_source))
assert no_of_records_in_source > 5, ".csv file does not have enough entries to perform useful comparisons"

Total number of records in source: 11122


In [5]:
# Check that the column names are as expected
expected_column_names = ['given_name', 'surname', 'date_of_birth', 'sex']
column_assert_message = "Input file has different column names from the Github sample"
for expected_column in expected_column_names:
    assert expected_column in df_people_records.columns, expected_column + " not found. " + column_assert_message

In [6]:
df_people_records.head()

,given_name,surname,date_of_birth,sex
0,mitchell,clausson,21/01/1980,m
1,thomas,skindstad,18/06/1988,m
2,harry,sodho,27/04/1983,f
3,jessica,haward,27/12/1992,f
4,maya,kurniawan,13/03/1992,f


## Requirement 2) the number of unique records in the source data set

In [7]:
# Duplicates are captured, and saved to a separate file, exact_duplicates.csv
df_duplicate_records = df_people_records[pd.DataFrame.duplicated(df_people_records)]
df_duplicate_records.to_csv('exact_duplicates.csv', index=False)

# Remove duplicates from current dataFrame
df_people_records = df_people_records.drop_duplicates()
number_of_unique_records = df_people_records.shape[0]
print("Number of unique records: " + str(number_of_unique_records))

Number of unique records: 10961


In [10]:
# Validate that all input data contains string or date information 
# The bulk of the date of birth data is in dd/mm/YYYY date format.
# However, a small number of entries were found with inconsistent dates.
date_format = "%d/%m/%Y"

def check_string(a_string):
    try:
        str(a_string)
    except InvalidOperation:
        return False
    return True

def check_date(a_date):
    try:
        datetime.datetime.strptime(a_date, date_format)
    except ValueError:
        return False
    return True

string_validation = [CustomElementValidation(lambda s: check_string(s), 'is not a string')]
date_validation = [CustomElementValidation(lambda d: check_date(d), 'is not a correct date')]

In [11]:
schema = pandas_schema.Schema([
            pandas_schema.Column('given_name', string_validation),
            pandas_schema.Column('surname', string_validation),
            pandas_schema.Column('date_of_birth', date_validation),
            pandas_schema.Column('sex', string_validation)])

In [12]:
# apply data type validation
errors = schema.validate(df_people_records)
errors_index_rows = [error.row for error in errors]

In [13]:
if len(errors_index_rows) > 0:
    df_invalid_data = df_people_records[df_people_records.index.isin(errors_index_rows)]
    print("Data in these rows did not match expected data type or format:")
    print(df_invalid_data)
    df_invalid_data.to_csv('invalid_data.csv', index=False)

Data in these rows did not match expected data type or format:
     given_name    surname date_of_birth sex
2193     daniel    rennoll    1987-12-32   m
2714     olivia  adolfsson    1995-12-32   f
8465       esme  muraguchi    1987-12-32   f
8582       kyle    siregar    2009-12-32   f
8978     brooke     strapp    1990-12-32   f


In [14]:
# New dataframe with any invalid data rows dropped
df_cleaned_records = df_people_records.drop(index=errors_index_rows)

In [16]:
# Unicode data found in data source - this makes comparison operations difficult.
# Convert unicode characters into their ascii equivalent equivalent characters
df_cleaned_records["ascii_given_name"] = df_cleaned_records['given_name'].apply(lambda x: str(unicodedata.normalize('NFKD', str(x)).encode('ascii', 'ignore')).split("'")[1])
df_cleaned_records["ascii_surname"] = df_cleaned_records['surname'].apply(lambda x: str(unicodedata.normalize('NFKD', str(x)).encode('ascii', 'ignore')).split("'")[1])

In [18]:
# Remove anything which is not a letter, e.g. hyphen, apostrophes, etc.
regex = re.compile('[^a-zA-Z]')
df_cleaned_records["ascii_given_name"] = df_cleaned_records["ascii_given_name"].apply(lambda x: regex.sub('', x))
df_cleaned_records["ascii_surname"] = df_cleaned_records["ascii_surname"].apply(lambda x: regex.sub('', x))

In [20]:
def check_for_similar_record(input_date):
    """
    check_for_similar_record - takes a date as input
    Retrieves all rows which include that date, and creates dictionaries of the given names and surnames.
    Fuzzywuzzy is used to score the similarity between the given names and the surnames.
    
    Any pair of row indexes which have a score of greater than 'fuzz_score' (set to 50, but this can be modified)
    for both the given name and the surname are considered to be similar.
    
    Output: row indices for the 2 records which are considered to be similar
    """
    given_names_dict = dict(enumerate(df_cleaned_records[df_cleaned_records['date_of_birth'] == input_date]['ascii_given_name']))
    surnames_dict = dict(enumerate(df_cleaned_records[df_cleaned_records['date_of_birth'] == input_date]['ascii_surname']))
    index_dict = dict(enumerate(df_cleaned_records[df_cleaned_records['date_of_birth'] == input_date].index))
    for this_record in range(len(surnames_dict)):
        for compare_record in range(len(surnames_dict)):
            # do not compare record against itself
            if not compare_record == this_record:
                given_name_score =  fuzz.ratio(given_names_dict[this_record], given_names_dict[compare_record])
                if given_name_score > fuzz_score:
                    surname_score =  fuzz.ratio(surnames_dict[this_record], surnames_dict[compare_record])
                    if surname_score > fuzz_score:
                        return index_dict[this_record], index_dict[compare_record]

In [21]:
# create a list of tuples, which contain the dataFrame indices for any matching record pairs
fuzzy_matched_records = [check_for_similar_record(x) for x in df_cleaned_records[df_cleaned_records.duplicated(subset=['date_of_birth'])]['date_of_birth'].unique()]

In [22]:
# Drop any empty entries from list
if len(fuzzy_matched_records) > 0:
    fuzzy_matched_records = [idx for idx in fuzzy_matched_records if idx]
    

In [23]:
# Create DataFrame of the fuzzy matched records 
if len(fuzzy_matched_records) > 0:
    df_fuzzy_matched_records = pd.concat([
        pd.DataFrame([df_cleaned_records.loc[idx[0]] for idx in fuzzy_matched_records]).reset_index(drop=True), 
        pd.DataFrame([df_cleaned_records.loc[idx[1]] for idx in fuzzy_matched_records]).reset_index(drop=True)], 
        axis=1, ignore_index=True)

In [25]:
# Drop Ascii versions of columns to leave the original encoding versions
df_fuzzy_matched_records.drop([4, 5, 10, 11], axis='columns', inplace=True)

In [26]:
# Rename fuzzy matched columns
df_fuzzy_matched_records.columns = ['given_name_A', 'surname_A', 'date_of_birth_A', 'sex_A', 
                                    'given_name_B', 'surname_B', 'date_of_birth_B', 'sex_B']

In [27]:
number_of_pairs_of_similar_records = df_fuzzy_matched_records.shape[0]

In [30]:
# Drop 2nd fuzzy match from original dataframe, and drop the ascii columns
df_cleaned_records.drop([idx[1] for idx in fuzzy_matched_records if idx], axis=0, inplace=True)
df_cleaned_records.drop(columns=['ascii_given_name', 'ascii_surname'], inplace=True)

In [31]:
df_cleaned_records.to_csv('cleaned.csv', index=False)

## Requirement 3) the number of different people in the source data set

In [32]:
number_of_different_people = df_cleaned_records.shape[0]
print("Number of different people: " + str(number_of_different_people))

Number of different people: 10120


## Requirement 4) Write the data that is considered duplicated/related to a CSV file called relateddata.csv

In [33]:
df_fuzzy_matched_records.to_csv('relateddata.csv', index=False)

In [34]:
# Create summary file
output_file = open("summary.txt", "w")
output_file.write("Summary information on the records in the input file: " + input_file + '\n')
output_file.write("====================================================================" + '\n\n')
output_file.write("Total number of records in source: " + str(no_of_records_in_source) + '\n')
output_file.write("Number of unique records: " + str(number_of_unique_records) + '\n')
output_file.write("Number of pairs of similar records: " + str(number_of_pairs_of_similar_records) + '\n')
output_file.write("Number of different people: " + str(number_of_different_people) + '\n')
output_file.close()